# Julia Tutorial

## PyData DC 2016

**Chase Coleman**

## What is needed to follow along

Before we get started, anyone who wants to follow along should do two things:

1. Get Julia by going to http://julialang.org/downloads/ and downloading the appropriate version of Julia
2. Download this notebook and other materials by "gitting" them from (TODO: add link once I've created a GH repo)

# What is Julia?

[Julia](http://julialang.org/) is a new(ish) programming language designed to be both high-level and high-performant.

The current release is 0.5.0.

## High-level

* <font color="green"> Prevents excess verbosity </font>
* <font color="green"> Simple syntax </font>
* <font color="green"> Dynamic types </font>

## (Potentially) High-performance

* <font color="green"> Compiled (JIT) </font>
* <font color="green"> Helps solve two language problem (Julia is mainly implemented in Julia) </font>

* <font color="red"> Sometimes naively written code suffers performance penalties (though this is true in all languages) </font>

## Hello World

In [1]:
println("Hello Julia World")

Hello Julia World


## Syntax Overview

Brief overview of basic Julia syntax

In [2]:
# Learn by example
function bisect(f::Function, a, b; maxiter=500, tol=1e-8)
    fa, fb = f(a), f(b)
    fa*fb > 0 ? error("f(a) and f(b) must have different signs") : nothing

    for i=1:maxiter
        c = (a+b)/2
        fc = f(c)

        abs(fc) > tol ? nothing : return (c, fc)

        if fa*fc > 0.0
            a = c
             fa = fc
        else
            b = c
            fb = fc
        end
    end

    error("Failed to converge in $maxiter iterations")
end

bisect (generic function with 1 method)

## Shorthand Function Definition

In [3]:
f(x::Float64) = x^3;
f1 = x -> x^3;

In [4]:
println(bisect(f, -2.0, 0.5))
println(bisect(f1, -2.0, 0.5))

(0.001953125,7.450580596923828e-9)
(0.001953125,7.450580596923828e-9)


## Comprehensions

In [5]:
x = collect(linspace(0, 2π, 50))
sin_x = [sin(i) for i in x];
println(sin_x[1:5])

[0.0,0.127877,0.253655,0.375267,0.490718]


## Broadcasting with matrices

Many matrix operations look like their Matlab counterparts

In [6]:
a = eye(2);  # Identity matrix
b = 2.0 * ones(2, 2);  # Matrix of all ones

println(a ./ b)  # elementwise divide
println(a .* b)  # elementwise multiply
println(a*b)  # matrix multiplication
println(a \ b[:, 1])  # solve a x = b
println(ones(2, 1) .+ ones(1, 3))  # Broadcasting tricks

[0.5 0.0; 0.0 0.5]
[2.0 0.0; 0.0 2.0]
[2.0 2.0; 2.0 2.0]
[2.0,2.0]
[2.0 2.0 2.0; 2.0 2.0 2.0]


## Vectorization

There is a relatively new `.` convention (mimics the notation for broadcasting) in Julia that transforms scalar functions into a vectorized function -- If you're interested in the details, see this github [issue](https://github.com/JuliaLang/julia/pull/17300)

In [8]:
f_x = f.(x)  # This function only knew how to operate on Float64s
sin_x = sin.(x)
println(sin_x[1:5])
println(f_x[1:5])

[0.0,0.127877,0.253655,0.375267,0.490718]
[0.0,0.00210839,0.0168671,0.0569266,0.134937]


# Types

Understanding Julia's type system will help write smart "Julian" code. Two "laws" of Julia:

* Everything in Julia is a type
* Every type in Julia has a supertype (think parent-child inheritance)

## Type Hierarchy Example

![alt text](./images/Type-hierarchy-for-julia-numbers.png "Type Hierarchy")

Image is from [wikibooks](https://en.wikibooks.org/wiki/Introducing_Julia/Types) and is licensed under [CC BY-SA](http://creativecommons.org/licenses/by-sa/4.0)

In [9]:
println(typeof(1.0))
println(typeof(1))
println(typeof(true))
println(typeof("A"))
println(typeof("ABC"))
println(typeof(Int8(10)))
println(typeof(ones(3)))

Float64
Int64
Bool
String
String
Int8
Array{Float64,1}


## Multiple Dispatch

**Disclaimer: I'm not a computer scientist, this is my understanding and it is possibly wrong.**

Object oriented languages, like Python, are by nature "single dispatch." Single dispatch is where the functions are specialized based on the type of their first argument -- Think about the `self` argument for class methods. Multiple dispatch is where functions are "specialized" based on the types of all of arguments.

To understand, let's see some examples.

In [10]:
# My dumb function
function mdf(a::Any, b::Any)
    println("The type of a is $(string(typeof(a)))")
    println("The type of b is $(string(typeof(b)))")

    return nothing
end

mdf (generic function with 1 method)

In [11]:
mdf(1, 1)
mdf(sum, 1//2)

The type of a is Int64
The type of b is Int64
The type of a is Base.#sum
The type of b is Rational{Int64}


In [12]:
mdf(a::Number, b) = println("a is a number")
mdf(a, b::Number) = println("b is a number")

function mdf(a::Number, b::Number)
    println("Ha! We're both numbers!!!")

    return nothing
end

mdf (generic function with 4 methods)

In [13]:
# Remember some of these returned something
# different before we defined a more specialized method
mdf(1, 1)
mdf(sum, 1//2)
mdf(0.0, sum)

Ha! We're both numbers!!!
b is a number
a is a number


## Type Parameters

Notice that below when we ask Julia the type of an array it gives us some additional information -- In particular, it tells us `Array{Float64, 1}`.

The information inside the curly brackets is referred to as "type parameters." Type parameters will allow us to further specialize our functions.

In [14]:
typeof(ones(3))

Array{Float64,1}

## User Types

# Package System

* Registered
* Unregistered

## Add and Remove Packages

## Plotting Packages

# Performance "gotchas"

## Type Stability

In [ ]:
function add_two_numbers(x::Float64, y::Float64)
    z = x + y
    return z
end

In [ ]:
@code_llvm add_two_numbers(1.0, 1.0)

In [ ]:
function my_sum_unstable(N::Int)
    total = 0
    for i=1:N
        total += i / N
    end

    return total
end

In [ ]:
function my_sum_stable(N::Int)
    total = 0.0
    for i=1:N
        total += i / N
    end

    return total
end

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark my_sum_stable(10_000_000)

In [ ]:
@benchmark my_sum_unstable(10_000_000)

In [ ]:
@code_warntype my_sum_stable(10)

In [ ]:
@code_warntype my_sum_unstable(10)